## 동영상 분석
- OpenCV를 통해 웹 카메라로부터의 입력 다루기

### 웹 카메라 이미지를 실시간으로 출력하기

In [2]:
import cv2
import numpy as np

# 웹 카메라로부터 입력 받기
cap = cv2.VideoCapture(0)
while True:
    # 카메라 이미지 읽어 들이기
    _, frame = cap.read()
    
    # 이미지 축소해서 출력하기
    frame = cv2.resize(frame, (480, 270))
    
    # 윈도우에 출력하기
    cv2.imshow('OpenCV Web Camera', frame)
    
    # ESC 또는 Enter 키 입력되면 종료
    k = cv2.waitKey(1) # 1msec 대기
    if k == 27 or k == 13:
        break
        
cap.release() # 카메라 해제
cv2.destroyAllWindows() # 윈도우 제거

### 카메라에서 붉은색 성분만 추출

In [3]:
import cv2
import numpy as np

# 웹 카메라로부터 입력 받기
cap = cv2.VideoCapture(0)
while True:
    # 이미지 추출
    _, frame = cap.read()
    
    # 이미지 축소
    frame = cv2.resize(frame, (480, 270))
    
    # 파란색, 녹색 없애기
    # Blue, Green, Red 순
    frame[:, :, 0] = 0 # Blue
    frame[:, :, 1] = 0 # Green
    
    # 윈도우에 출력하기
    cv2.imshow('Red Camera', frame)
    
    # ESC 또는 Enter 키 입력되면 종료
    k = cv2.waitKey(1) # 1msec 대기
    if k == 27 or k == 13:
        break
        
cap.release() # 카메라 해제
cv2.destroyAllWindows() # 윈도우 제거

### HSV 색공간을 사용해 색 검출하기
- Hue(색상), 붉은색>녹색>파란색>붉은색의 360도 원형으로 표현
- Saturation(채도)
- Value Brightness(명도)

[HSV 색공간](https://ko.wikipedia.org/wiki/HSV_%EC%83%89_%EA%B3%B5%EA%B0%84)

In [6]:
import cv2
import numpy as np

# 웹 카메라로부터 입력 받기
cap = cv2.VideoCapture(0)
while True:
    # 이미지 추출
    _, frame = cap.read()
    
    # 이미지 축소
    frame = cv2.resize(frame, (480, 270))
    
    # 색공간을 HSV로 변환
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV_FULL)
    
    # HSV 분할
    h = hsv[:, :, 0]
    s = hsv[:, :, 1]
    v = hsv[:, :, 2]
    
    # 붉은색 가진 요소만 출력
    img = np.zeros(h.shape, dtype=np.uint8)
    img[((h < 50) | (h > 200)) & (s > 100)] = 255
    
    # 윈도우에 출력하기
    cv2.imshow('Red Camera', img)
    
    # ESC 또는 Enter 키 입력되면 종료
    k = cv2.waitKey(1) # 1msec 대기
    if k == 27 or k == 13:
        break
        
cap.release() # 카메라 해제
cv2.destroyAllWindows() # 윈도우 제거

## 화면에 움직임이 있는 부분 추출하기
- `cv2.absdiff()` 함수를 사용하여 이미지의 차이를 확인

In [7]:
import cv2

cap = cv2.VideoCapture(0)
img_last = None # 이전 프레임을 저장해둘 변수
green = (0, 255, 0)

while True:
    # 이미지 추출
    _, frame = cap.read()
    frame = cv2.resize(frame, (480, 270))
    
    # 흑백 이미지로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9, 9), 0)
    img_b = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)[1]
    
    # 차이 확인하기
    if img_last is None:
        img_last = img_b
        continue
    
    frame_diff = cv2.absdiff(img_last, img_b)
    cnts = cv2.findContours(frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    # 차이가 있는 부분 출력하기
    for pt in cnts:
        x, y, w, h = cv2.boundingRect(pt)
        if w < 30 : continue
        cv2.rectangle(frame, (x, y), (x+w, y+h), green, 2)
    
    # 프레임을 변수에 저장
    img_last = img_b
    
    # 화면에 출력
    cv2.imshow('Diff Camera', frame)
    cv2.imshow('diff data', frame_diff)
    if cv2.waitKey(1) == 13: break
cap.release()
cv2.destroyAllWindows()

## 이미지 파일 쓰기
- `cv2.VideoWriter(filename, format, fps, size)` : 동영상 쓰기 전용 객체 생성
    - `format` : 동영상 형식 지정
        - `cv.VideoWriter_fourcc('m', 'p', '4', 'v')` : MPEG-4 Video를 나타내는 mp4v를 한 글자씩 지정

In [9]:
import cv2
import numpy as np

# 카메라 입력 받기
cap = cv2.VideoCapture(0)

# 동영상 출력 전용 객체 생성
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
fps = 20.0
size = (1920, 1080)
writer = cv2.VideoWriter('../datasets/test.m4v', fmt, fps, size)

while True:
    _, frame = cap.read()
    frame = cv2.resize(frame, size)
    
    # 이미지 출력 (영상에 저장)
    writer.write(frame)
    
    # 화면에도 출력
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) == 13:break

writer.release()
cap.release()
cv2.destroyAllWindows()

## 동영상에서 열대어가 나오는 부분 검출하기

In [13]:
import cv2, os

img_last = None # 이전 프레임 저장할 변수
no = 0 # 이미지 장수
save_dir = '../datasets/exfish' # 저장 디렉터리
os.mkdir(save_dir) # 디렉터리 생성

# 동영상 파일로부터 입력 받기
cap = cv2.VideoCapture('../datasets/fish.mp4')
while True:
    # 이미지 추출
    is_ok, frame = cap.read()
    if not is_ok : break
    frame = cv2.resize(frame, (640, 360))
    
    # 흑백 이미지로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (15, 15), 0)
    img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)[1]
    
    # 차이 확인
    if not img_last is None:
        frame_diff = cv2.absdiff(img_last, img_b)
        cnts = cv2.findContours(frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        
        # 차이가 있는 부분 파일로 출력
        for pt in cnts:
            x, y, w, h = cv2.boundingRect(pt)
            if w < 100 or x > 500 : continue # 노이즈 제거
            # 추출한 영역 저장
            imgex = frame[y:y+h, x:x+w]
            outfile = save_dir +'/' + str(no) + '.jpg'
            cv2.imwrite(outfile, imgex)
            no += 1
    img_last = img_b
cap.release()
print('ok')

ok


## 머신러닝으로 동영상에서 열대어가 많이 나오는 부분 찾기
- 열대어가 나오는 이미지와 없는 이미지 분리하여 fish, nofish 디렉터리에 분류

In [15]:
import cv2
import os, glob
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# 이미지 학습 크기와 경로 지정
image_size = (64, 32)
path_fish = '../datasets/fish'
path_nofish = '../datasets/nofish'
x = [] # 이미지 데이터
y = [] # 레이블 데이터

# 이미지를 읽어 들이고 배열에 넣기
def read_dir(path, label):
    files = glob.glob(path + '/*.jpg')
    for f in files:
        img = cv2.imread(f)
        img = cv2.resize(img, image_size)
        img_data = img.reshape(-1, ) # 1차원으로 전개
        x.append(img_data)
        y.append(label)

# 이미지 데이터 읽어 들이기
read_dir(path_nofish, 0)
read_dir(path_fish, 1)

# train, test 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# 데이터 학습
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

# 정답률 확인
y_pred = clf.predict(x_test)
print('정확도 :', accuracy_score(y_test, y_pred))

# 데이터 저장
joblib.dump(clf, '../models/fish.pkl')

정확도 : 0.9054593874833555


['../models/fish.pkl']

### 동영상 분석하기
- 열대어가 많이 나오는 부분을 추출

In [17]:
import cv2, os, copy, joblib

# 학습한 데이터 읽어 들이기
clf = joblib.load('../models/fish.pkl')
output_dir = '../datasets/bestshot'
img_last = None
fish_th = 3 # 이미지로 출력할 기준이 되는 물고기 수
count = 0
frame_count = 0
if not os.path.isdir(output_dir) : os.mkdir(output_dir)

# 동영상 파일 읽어 들이기
cap = cv2.VideoCapture('../datasets/fish.mp4')
while True:
    # 이미지 추출
    is_ok, frame = cap.read()
    if not is_ok:break
    frame = cv2.resize(frame, (640, 360))
    frame2 = copy.copy(frame)
    frame_count += 1
    
    # 이전 프레임과 비교를 위해 흑백으로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (15, 15), 0)
    img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)[1]
    
    if not img_last is None:
        # 차이 추출
        frame_diff = cv2.absdiff(img_last, img_b)
        cnts = cv2.findContours(frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        
        # 차이가 있는 부분에 물고기가 있는지 확인
        fish_count = 0
        for pt in cnts:
            x, y, w, h = cv2.boundingRect(pt)
            if w < 100 or x > 500 : continue # 노이즈 제거
            # 추출한 영역에 물고기가 있는지 확인
            imgex = frame[y:y+h, x:x+w]
            imagex = cv2.resize(imgex, (64, 32))
            img_data = imagex.reshape(-1, )
            pred_y = clf.predict([img_data])
            if pred_y[0] == 1:
                fish_count += 1
                cv2.rectangle(frame2, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
        # 물고기가 많이 있는지 확인
        if fish_count > fish_th:
            fname = output_dir + '/fish' + str(count) + '.jpg'
            cv2.imwrite(fname, frame)
            count += 1
            
    cv2.imshow('FISH!', frame2)
    if cv2.waitKey(1) == 13:break
    img_last = img_b
cap.release()
cv2.destroyAllWindows()
print('ok', count, '/', frame_count)

ok 48 / 1990
